# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы изучите методы работы с категориальными переменными

In [2]:
%pylab inline
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(2 балла)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [3]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [4]:
from sklearn.model_selection import train_test_split
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию, найдтие оптимум. Сделайте предсказание на тестовой части выборки. Замерьте качество.

In [6]:
def print_k(y_pred, y_test):
    accuracy = accuracy_score(y_pred, y_test)
    precision = precision_score(y_pred, y_test)
    recall = recall_score(y_pred, y_test)
    f1 = f1_score(y_pred, y_test)
    
    print(f"accuracy: {accuracy:.4f}")
    print(f"precision: {precision:.4f}")
    print(f"recall: {recall:.4f}")
    print(f"f1: {f1:.4f}")

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = LogisticRegression(C=1.0, solver='liblinear', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print_k(y_pred, y_test)

accuracy: 0.5882
precision: 0.5486
recall: 0.5958
f1: 0.5712


__Выводы__ в свободной форме:

In [8]:
# Качество модели не очень высокое, однако в большинстве случаев модель классифицирует верно

__Задание 2.__ Изучение влияния регуляризатора на процесс обучения

__(2 балла)__

Проверьте на практике, как влияет регуляризатор на процесс обучения (убывание функции потерь на обучающей и отложенной выборках). Чтобы считать функцию потерь на отложенной выборке после каждой итерации, запускайте процесс обучения логистической регрессии с параметром $max\_iter=1$ и $w^{(0)}$, полученным на предыдущей итерации. Постройте два графика: на одном из них логистическая регрессия с коэффициентом регуляризации, равным 0, а на другом с некоторым разумным значением. На каждом графике одновременно должна быть и функция потерь для обучающей, и для тестовой выборки. Не забудьте сделать одинаковыми оси обоих графиков. Какие выводы вы можете сделать?

In [9]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

def track_logistic_regression_loss(X_train, X_test, y_train, y_test, C=1.0, max_iter=1000):
    train_losses = []
    test_losses = []
    
    model = LogisticRegression(C=C, max_iter=1, warm_start=True, 
                              random_state=42, solver='lbfgs')
    
    for i in range(1, max_iter + 1):
        model.max_iter = i
        model.fit(X_train, y_train)
        
        # Предсказания вероятностей
        y_train_pred_proba = model.predict_proba(X_train)
        y_test_pred_proba = model.predict_proba(X_test)
        
        # Вычисляем потери
        train_loss = log_loss(y_train, y_train_pred_proba)
        test_loss = log_loss(y_test, y_test_pred_proba)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
    return train_losses, test_losses


train_losses_no_reg, test_losses_no_reg = track_logistic_regression_loss(
    X_train_scaled, X_test_scaled, y_train, y_test, C=1.0, max_iter=1000)

train_losses_reg, test_losses_reg = track_logistic_regression_loss(
    X_train_scaled, X_test_scaled, y_train, y_test, C=0.01, max_iter=1000)


# Строим графики
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

#  С = 0
iterations = range(1, len(train_losses_no_reg) + 1)
ax1.plot(iterations, train_losses_no_reg, label='Обучающая выборка', linewidth=2)
ax1.plot(iterations, test_losses_no_reg, label='Тестовая выборка', linewidth=2)
ax1.set_title('Без регуляризации (C=1.0)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Итерация')
ax1.set_ylabel('Log Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.0)

С = 0.01
ax2.plot(iterations, train_losses_reg, label='Обучающая выборка', linewidth=2)
ax2.plot(iterations, test_losses_reg, label='Тестовая выборка', linewidth=2)
ax2.set_title('С регуляризацией (C=0.01)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Итерация')
ax2.set_ylabel('Log Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.0)

plt.tight_layout()
plt.show()


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

KeyboardInterrupt: 

__Выводы:__

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [10]:
from sklearn.preprocessing import OneHotEncoder
import time


data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

In [12]:
cat_features = [col for col in X_train.columns if 'cat' in col]
num_features = [col for col in X_train.columns if col not in cat_features]

onehot_encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

X_train_cat_encoded = onehot_encoder.fit_transform(X_train[cat_features])
X_test_cat_encoded = onehot_encoder.transform(X_test[cat_features])

encoded_feature_names = onehot_encoder.get_feature_names_out(cat_features)

X_train_encoded_df = pd.DataFrame(
    X_train_cat_encoded, 
    columns=encoded_feature_names, 
    index=X_train.index
)
X_test_encoded_df = pd.DataFrame(
    X_test_cat_encoded, 
    columns=encoded_feature_names, 
    index=X_test.index
)


X_train_combined = pd.concat([X_train[num_features], X_train_encoded_df], axis=1)
X_test_combined = pd.concat([X_test[num_features], X_test_encoded_df], axis=1)


start = time.time()

onehot_model = LogisticRegression(random_state=42, max_iter=2000, n_jobs=-1)
onehot_model.fit(X_train_combined, y_train)

onehot_train_time = time.time() - start


y_pred_onehot = onehot_model.predict(X_test_combined)


print(f"Время обучения модели: {onehot_train_time:.4f} секунд")
print("\nМодель c one-hot:") 
print_k(y_pred_onehot, y_test)
print("\nМодели без one-hot:")
print_k(y_pred, y_test)

Время обучения модели: 16.6489 секунд

Модель c one-hot:
accuracy: 0.5945
precision: 0.5623
recall: 0.6011
f1: 0.5810

Модели без one-hot:
accuracy: 0.5882
precision: 0.5486
recall: 0.5958
f1: 0.5712


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

__Вывод:__

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [ ]:
# Your code here
# ...

__Вывод:__